**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

# Generator Evaluation Metrics

![](https://i.imgur.com/GaMHy7w.png)

The generation step, which comes after retrieval, generally includes:

- Building a prompt that combines the initial input with the context retrieved in the previous step.
- Feeding this prompt to your LLM, which produces the final generated response.


Key Metrics to Evaluate here include:

- Answer Relevancy
- Faithfulness
- Hallucination Check
- Custom LLM as a Judge (G-Eval)

## LLM-based Answer Relevancy - DeepEval

The answer relevancy metric measures the quality of your RAG pipeline's generator by evaluating how relevant the `actual_output` of your LLM application is compared to the provided `input`.

`deepeval`'s answer relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the AnswerRelevancyMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query
- `actual_output` : Actual LLM Response


![](https://i.imgur.com/GbNSCFC.png)



## Semantic Similarity based Answer Relevancy - RAGAS

DeepEval has bindings to Ragas which enables us to use the RAGASAnswerRelevancyMetric which focuses on assessing how pertinent the generated answer is to the given query using cosine similarity. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy.

![](https://i.imgur.com/vq1ytZ3.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example - DeepEval:

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
)

metric = AnswerRelevancyMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

### Example - RAGAS:

In [0]:
from deepeval.metrics.ragas import RAGASAnswerRelevancyMetric

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
)

metric = RAGASAnswerRelevancyMetric(
    threshold=0.5,
    model="gpt-4o",
    embeddings=OpenAIEmbeddings()
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

## Faithfulness

The faithfulness metric measures the quality of your RAG pipeline's generator by evaluating whether the `actual_output` factually aligns with the contents of your `retrieval_context`.

`deepeval`'s faithfulness metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the FaithfulnessMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/OCSFPTb.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import FaithfulnessMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    retrieval_context=retrieved_context
)

metric = FaithfulnessMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

## Hallucination Check

The hallucination metric determines whether your LLM generates factually correct information by comparing the `actual_output` to the provided (human ground truth) `context`.

`deepeval`'s hallucination metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the HallucinationMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response
- `context` : Human Ground Truth Context Document Chunks (Nodes)


![](https://i.imgur.com/qyVBKU2.png)





In [0]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

### Example:

In [0]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

In [0]:
human_ground_truth_context = ["Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
                              "Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition."]
human_ground_truth_context

In [0]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import HallucinationMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    context=human_ground_truth_context
)

metric = HallucinationMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

In [0]:
ai_response = 'AI refers to machines mimicking human intelligence to produce cyborgs and electric sheep'

In [0]:
test_case = LLMTestCase(
    input=response['question'],
    actual_output=ai_response,
    context=human_ground_truth_context
)

metric = HallucinationMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

In [0]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)